<div >
<img src = "../banner.jpg" />
</div>

<a target="_blank" href="https://colab.research.google.com/github/ignaciomsarmiento/BDML_202402/blob/main/Lecture10/Notebook_Boosting.ipynb">
  <img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/>
</a>


# Tree-Based Methods

## Boosting Trees

Let's load the packages:

In [ ]:
# install.packages("pacman") #run this line if you use Google Colab

In [ ]:
#packages
require("pacman")
p_load("tidyverse","ggplot2","rpart")

In [ ]:


db<-read.csv("https://raw.githubusercontent.com/ignaciomsarmiento/datasets/main/boosting_tree_toy.csv")
head(db)

y<-db$y
x<-db$x

In [ ]:
plot(x,y)

### Hiperparámetros

- $\lambda$ la tasa a la que aprende, los valores típicos son 0.01 o 0.001

In [ ]:
lambda<-.01

- Tamaño del árbol. Arboles pocos profundos  funcionan bien.

In [ ]:
d<-1 #stump

- Iniciamos fijando $\hat{f}(x)=0$ y $r_i=y_i$ para todos los $i$ del training set

In [ ]:
fhat=rep(0,length(y))

r=y

In [ ]:
head(y)

In [ ]:
head(r)

Para $m=1,2,...,M$

 - Ajustamos un árbol $\hat{f}^m$ con $d$ bifurcaciones ($d+1$ hojas)

In [ ]:
#Primera iteración
fit1<-rpart(r ~ x, control = list(maxdepth = d))
yhat1<-predict(fit1,newdata=data.frame(x))

head(lambda *yhat1)

   - Actualizamos $\hat{f}(x)$ con una versión "shrunken" del nuevo árbol

In [ ]:
f1<-fhat + lambda *yhat1

In [ ]:
plot(x,y,ylab="",xlab="")
lines(x,f1,type="s",col="red",lwd=3)

- Actualizamos los residuales
  \begin{align}
  r_i\leftarrow r_i-\lambda\hat{f}^m(x)
  \end{align}

In [ ]:
r1<- r - lambda*yhat1

El loop vuelve a iniciar, en la iteración 2:

In [ ]:
# iteracion 2

fit2<-rpart(r1 ~ x, control = list(maxdepth = d))
yhat2<-predict(fit2,newdata=data.frame(x))
f2<- f1 + lambda *yhat2

head(lambda *yhat2)

In [ ]:
plot(x,y,ylab="",xlab="")
lines(x,f2,type="s",col="red",lwd=3)

In [ ]:
head(f2)

In [ ]:
# En un loop
fhat<-rep(0,length(y))

r = y

YP<-lambda*fhat

for(t in 1:500){
  fit <- rpart(r~x, control = list(maxdepth = 1))
  yhat<- predict(fit,newdata=data.frame(x))
  r <-  r - lambda*yhat
  YP <-  cbind(YP,lambda*yhat)}

head(YP)

In [ ]:
sum(YP[1,c(1,2,3)])

In [ ]:
M<-500

fhat<-apply(YP[,1:M],1,sum)

plot(x,y,ylab="",xlab="")
lines(x,fhat,type="s",col="red",lwd=3)

## Predicting House Prices


$$
Precio=f(structural\,attributes,location,...)
$$



### Ames Data Set

The data set contains information from the Ames Assessor’s Office used in computing assessed values for individual residential properties sold in Ames, IA from 2006 to 2010.

In [ ]:
p_load("modeldata","caret")

data("ames", package = "modeldata")

In [ ]:
dim(ames)

In [ ]:
head(ames)

The description of the variables can be viewed here: https://jse.amstat.org/v19n3/decock/DataDocumentation.txt

### Gradient Boosting Trees

\begin{align}
\hat{f}_{gbm}(x)=\sum_{m=1}^{M}\lambda \hat{f}_{m}(x_{i})
\end{align}

- n.trees (# Boosting Iterations) $M$
- interaction.depth (Max Tree Depth) $J$
- shrinkage (Shrinkage) $\lambda$
- n.minobsinnode (Min. Terminal Node Size) 

In [ ]:
p_load('gbm')


grid_gbm<-expand.grid(n.trees=c(200,300,500),
                      interaction.depth=c(4,6),
                      shrinkage=c(0.001,0.01),
                      n.minobsinnode = c(10,30))


grid_gbm


In [ ]:
fitControl<-trainControl(method ="cv",
                         number=5)

set.seed(1011)
gbm_tree <- train( log(Sale_Price) ~ .,
    data=ames,
  method = "gbm", 
  trControl = fitControl,
  tuneGrid=grid_gbm,
  verbose = FALSE
)            


In [ ]:
gbm_tree

In [ ]:
min(gbm_tree$results$RMSE)

### Xgboost


\begin{align}
\mathcal{L} &= \sum_{i=1}^N L(y_i,\hat{y}_i) + \sum_{k=1}^m \Omega(f_k)
\end{align}


In [ ]:
p_load('xgboost')

Hiper parametros en `R`

-  nrounds (# of trees): This parameter is similar to the random forest parameter. It determines the number of trees in the gradient-boosting forest (M). A range I often explore  100 and 1000 trees.

- max_depth (Max Tree Depth): The depth of a tree corresponds to how many branches deep each tree is allowed to go (J). A larger depth corresponds to more flexible trees, which can also lead to overfitting. The default tree depth in xgboost is 6, and it is often useful to explore max_depth range of between 4 and 8.

- eta (Shrinkage): The eta parameter controls how the boosted tree at a given iteration is merged with the tree in the previous iteration ($\eta$ in the algorithm). The default value is 0.3, and evaluating a range of effects between 0.01 and 0.5 is often important.

- min_child_weight (min obs per node) This parameter refers to the minimum allowable number of observations in each node. The default value is 1. However, this number is quite low, and caution is warranted here, particularly when the max_depth is large. I always set this value to at least ten and often explore ranges between 10 and 50.

- gamma (Minimum Loss Reduction) The gamma parameter controls the extent to which changes in the tree structure occur based on changes in the loss function (this is the $\gamma$ in the equation). The default gamma value is 0.  I often explore 0 and 1

- lambda (regularization term on weights) is the $\lambda$ parameter in the above equation. The default is set to 1.

- colsample_bytree (Subsample Ratio of Columns)
 
- subsample (Subsample Percentage)



In [ ]:
grid_xbgoost <- expand.grid(nrounds = c(100,250),
                            max_depth = c(2,4), 
                            eta = c(0.01,0.05), 
                            gamma = c(0,1), 
                            min_child_weight = c(10, 25),
                            colsample_bytree = c(0.33,0.66),
                            subsample = c(0.4,0.8))

grid_xbgoost

In [ ]:
set.seed(1011)
Xgboost_tree <- train(log(Sale_Price) ~ .,
    data=ames,
  method = "xgbTree", 
  trControl = fitControl,
  tuneGrid=grid_xbgoost
)        

Xgboost_tree

In [ ]:
Xgboost_tree$bestTune

In [ ]:
min(Xgboost_tree$results$RMSE)

It’s important to note that there is never a single optimal tuning parameter choice for all (or even most) settings. In fact, the choice of a given tuning parameter often has important effects on the optimal value of other tuning parameters. There are usually important tradeoffs to consider between tuning parameter values. For example, in the context of XGBoost, a high value of `min_child_weight` may require a lower `eta` or `max_depth` value. However, this is true for other types of machine learning algorithms.